<a href="https://colab.research.google.com/github/jihun212/HAICON2021/blob/main/%EC%B5%9C%EC%A7%80%ED%9B%882.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**아래에 나오는 모든 문자열이 UTF-8 형식이라는 것을 명시하는 코드**

In [ ]:
#-*- coding: utf-8 -*-

## Import Library

필요한 library들을 import 합니다.

In [ ]:
import os, sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from datetime import timedelta
import dateutil
from tqdm import tqdm
import easydict
from TaPR_pkg import etapr

## 데이터 전처리

제공된 csv 데이터를 읽어 오는 작업을 진행합니다.

In [ ]:
def dataframe_from_csv(target):
    return pd.read_csv(target).rename(columns=lambda x: x.strip())

In [ ]:
def dataframe_from_csvs(targets):
    return pd.concat([dataframe_from_csv(x) for x in targets])

normalize 함수는 Dataframe을 정규화합니다. 정규화 방법은 최솟값, 최댓값을 이용하여 0~1의 범위에 들어오도록 하는 것입니다.

가끔 값이 전혀 변하지 않는 필드가 있습니다. 이 경우 최솟값과 최댓값이 같을 것입니다. 본 문서에서는 이런 필드를 모두 0으로 만들었습니다.

In [ ]:
def normalize(df):
    ndf = df.copy()
    for c in df.columns:
        if TAG_MIN[c] == TAG_MAX[c]:
            ndf[c] = df[c] - TAG_MIN[c]
        else:
            ndf[c] = (df[c] - TAG_MIN[c]) / (TAG_MAX[c] - TAG_MIN[c])
    return ndf

### Feature Selection

각 columns 별 데이터의 분포를 시각화하여 확인합니다. 몇몇 데이터는 유의미하지 않기 때문에, 추후 feature selection을 진행해서 모델 효율성을 증진시킬 수 있었습니다.

In [ ]:
for i, v in enumerate(TEST_DF.columns.values):
    fig= plt.figure()
    plt.plot(TEST_DF[v])
    MIN = TEST_DF[v].min()
    MAX = TEST_DF[v].max()
    plt.plot(   (TEST_DF_RAW[ATTACK_FIELD]*MAX + MIN)  )
    fig.savefig('./data/sensor_graph_test/'+v)

#학습 모델 설정 & 데이터 입출력 정의

딥러닝 학습과 추론에는 PyTorch를 사용했습니다.

정상 데이터만 학습해야 하고, 정상 데이터에는 어떠한 label도 없으므로 unsupervised learning을 해야 합니다.


모델의 입출력은 다음과 같이 설정했습니다.

입력 : 윈도우에 해당하는 값

출력 : 윈도우의 가장 마지막 초(입력+1)의 값

이후 탐지 시에는 모델이 출력하는 값(예측값)과 실제로 들어온 값의 차를 보고 차이가 크면 이상으로 간주했습니다. 많은 오차가 발생한다는 것은 기존에 학습 데이터셋에서 본 적이 없는 패턴이기 때문이라는 가정입니다.

In [ ]:
class HS_DATASET(Dataset):
    def __init__(self, timestamps, df, window_size=90, stride=1, attacks=None):
        window_given=window_size - 1
        self.ts = np.array(timestamps)
        self.tag_values = np.array(df, dtype=np.float32)
        self.valid_idxs = []
        self.window_size = window_size
        self.window_given = window_given
        for L in tqdm(range(len(self.ts) - self.window_size + 1)):
            R = L + self.window_size - 1
            if dateutil.parser.parse(self.ts[R]) - dateutil.parser.parse(
                self.ts[L]
            ) == timedelta(seconds=self.window_size - 1):
                self.valid_idxs.append(L)
        self.valid_idxs = np.array(self.valid_idxs, dtype=np.int32)[::stride]
        self.n_idxs = len(self.valid_idxs)
        print(f"# of valid windows: {self.n_idxs}")
        if attacks is not None:
            self.attacks = np.array(attacks, dtype=np.float32)
            self.with_attack = True
        else:
            self.with_attack = False

    def __len__(self):
        return self.n_idxs

    def __getitem__(self, idx):
        i = self.valid_idxs[idx]
        last = i + self.window_size - 1
        item = {"attack": self.attacks[last]} if self.with_attack else {}
        item["ts"] = self.ts[i + self.window_size - 1]
        item["given"] = torch.from_numpy(self.tag_values[i : i + self.window_given])
        item["answer"] = torch.from_numpy(self.tag_values[last])
        return item

# 모델 구조 설정

**model 정의**

In [ ]:
class model(nn.Module):
    def __init__(self, input_size, window_size):
        super().__init__()
        self.layer1 = nn.Conv1d(in_channels=input_size, out_channels = input_size*20, kernel_size = 13, stride = 1, groups = input_size) #1D CNN Layer
        self.relu = nn.ReLU() #ReLU activation function
        self.ap1 = nn.AvgPool1d(3, stride=1) #Pooling Layer
        self.layer2 = nn.Conv1d(in_channels = input_size*20, out_channels=input_size*20*3, kernel_size = 1, stride = 1) #1D CNN Layer
        self.fc = nn.Linear((window_size-1-12-2)*input_size*20*3, input_size) #Fully Connected Layer

    def forward(self, in_x):
        in_x = torch.transpose(in_x, 1, 2)
        x = self.layer1(in_x) #1D CNN Layer
        x = self.relu(x) #ReLU activation function
        x = self.ap1(x) #Pooling Layer
        x = self.layer2(x) #1D CNN Layer
        x = self.relu(x) #ReLU activation function
        x = torch.flatten(x, start_dim=1) #Flatten Layer
        out = self.fc(x) #Fully Connected Layer

        return out

###Training을 위해 Loss function과 Metric을 선정했습니다.

Loss function은 MSE를 선택했고, optimizer는 AdamW(Loshchilov & Hutter, "Decoupled Weight Decay Regularization", ICLR 2019)를 사용합니다.

In [ ]:
def train(dataset, model, batch_size, n_epochs):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    optimizer = torch.optim.AdamW(model.parameters())
    loss_fn = torch.nn.MSELoss()
    epochs = tqdm(range(n_epochs), total=n_epochs, desc="training")
    best = {"loss": sys.float_info.max}
    loss_history = []
    for e in epochs:
        epoch_loss = 0
        for batch in tqdm(dataloader, total=len(dataloader), desc="batch"):
            optimizer.zero_grad()
            given = batch["given"].to(args.device)
            guess = model(given)
            answer = batch["answer"].to(args.device)
            loss = loss_fn(answer, guess)
            loss.backward()
            epoch_loss += loss.item()
            optimizer.step()
        loss_history.append(epoch_loss)
        epochs.set_postfix_str(f"loss: {epoch_loss:.6f}")
        if epoch_loss < best["loss"]:
            best["state"] = model.state_dict()
            best["loss"] = epoch_loss
            best["epoch"] = e + 1
    return best, loss_history

**inference function을 정의하고 validation dataset을 추론합니다.**

In [ ]:
def inference(dataset, model, batch_size):
    dataloader = DataLoader(dataset, batch_size=batch_size)
    ts, dist, att = [], [], []
    with torch.no_grad():
        for batch in dataloader:
            given = batch["given"].to(args.device)
            answer = batch["answer"].to(args.device)
            guess = model(given)
            ts.append(np.array(batch["ts"]))
            dist.append(torch.abs(answer - guess).cpu().numpy())
            try:
                att.append(np.array(batch["attack"]))
            except:
                att.append(np.zeros(batch_size))
    return (
        np.concatenate(ts),
        np.concatenate(dist),
        np.concatenate(att),
    )

모델에서 생성된 anomaly score를 시각화할 수 있습니다.

In [ ]:
def check_graph(xs, att, label, piece=2, THRESHOLD=None, FILENAME='result'):
    l = xs.shape[0]
    chunk = l // piece
    fig, axs = plt.subplots(piece + 1, figsize=(20, 4 * piece))
    for i in range(piece):
        L = i * chunk
        R = min(L + chunk, l)
        xticks = range(L, R)
        axs[i].plot(xticks, xs[L:R])
        if len(xs[L:R]) > 0:
            peak = max(xs[L:R])
            axs[i].plot(xticks, att[L:R] * peak * 0.3)
        if THRESHOLD!=None:
            axs[i].axhline(y=THRESHOLD, color='r')

    loss_normal_list = np.zeros(l)
    cnt_normal = 0
    loss_anomaly_list = np.zeros(l)
    cnt_anomaly = 0
    for i, loss in enumerate(xs):
        if label[i] == 0:
            loss_normal_list[cnt_normal] = loss
            cnt_normal += 1
        else:
            loss_anomaly_list[cnt_anomaly] = loss
            cnt_anomaly += 1
    loss_normal_list = loss_normal_list[0:cnt_normal]
    loss_anomaly_list = loss_anomaly_list[0:cnt_anomaly]

    fig.savefig(FILENAME)

In [ ]:
def put_labels(distance, threshold):
    xs = np.zeros_like(distance)
    xs[distance > threshold] = 1
    return xs

window 방식으로 추론이 진행됐기 때문에, 처음 시작 부분과 데이터셋 중간에 시간이 연속되지 않는 부분을 0으로 채워주기 위한 함수를 정의합니다.

In [ ]:
def fill_blank(check_ts, labels, total_ts):
    def ts_generator():
        for t in total_ts:
            yield dateutil.parser.parse(t)

    def label_generator():
        for t, label in zip(check_ts, labels):
            yield dateutil.parser.parse(t), label

    g_ts = ts_generator()
    g_label = label_generator()
    final_labels = []

    try:
        current = next(g_ts)
        ts_label, label = next(g_label)
        while True:
            if current > ts_label:
                ts_label, label = next(g_label)
                continue
            elif current < ts_label:
                final_labels.append(0)
                current = next(g_ts)
                continue
            final_labels.append(label)
            current = next(g_ts)
            ts_label, label = next(g_label)
    except StopIteration:
        return np.array(final_labels, dtype=np.int8)

In [ ]:
TRAIN_DATASET = sorted([x for x in Path("./data/train/").glob("*.csv")])
TEST_DATASET = sorted([x for x in Path("./data/test/").glob("*.csv")])
VALIDATION_DATASET = sorted([x for x in Path("./data/validation/").glob("*.csv")])

In [ ]:
TIMESTAMP_FIELD = "timestamp"
IDSTAMP_FIELD = 'id'
ATTACK_FIELD = "attack"

In [ ]:
TRAIN_DF_RAW = dataframe_from_csvs(TRAIN_DATASET)
VALIDATION_DF_RAW = dataframe_from_csvs(VALIDATION_DATASET)
TEST_DF_RAW = dataframe_from_csvs(TEST_DATASET)

최종 모델에서는 33개의 Column을 선택하여 모델을 학습했습니다. 더 많은 Column을 포함해서 학습해 보았지만, public score 기준으로 성능이 낮게나와 최종 모델로 선정하지 않았습니다.

In [ ]:
DROP_FIELD = ["timestamp", "C01", "C02", "C03", "C04", "C06", "C07", "C08", "C09", "C10", "C11", "C13", "C17",
                "C18", "C19", "C20", "C22", "C25", "C26", "C29", "C30", "C33", "C34", "C35", "C36", "C37",
                "C38", "C39", "C41", "C42", "C44", "C45", "C46", "C48", "C49", "C50", "C52", "C53", "C55",
                "C58", "C60", "C61", "C63", "C64", "C65", "C66", "C69", "C79", "C81", "C82",
                "C83", "C84", "C85", "C86"]

VALID_COLUMNS_IN_TRAIN_DATASET = TRAIN_DF_RAW.columns.drop(DROP_FIELD)

TAG_MIN과 TAG_MAX는 학습 데이터셋에서 최솟값 최댓값을 얻은 결과입니다.

In [ ]:
TAG_MIN = TRAIN_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET].min()
TAG_MAX = TRAIN_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET].max()

**hyperparameter 선정**

In [ ]:
args = easydict.EasyDict({
    "batch_size": 256, ## 배치 사이즈 설정  
    "device": torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'), ## GPU 사용 여부 설정
    "input_size": len(VALID_COLUMNS_IN_TRAIN_DATASET), ## 입력 차원 설정
    "output_size": len(VALID_COLUMNS_IN_TRAIN_DATASET), ## 출력 차원 설정
    "window_size" : 90, ## sequence Length
    "epochs" : 256, ## epoch 사이즈 설정
})

MODEL 경로명 설정 및 저장 설정

In [ ]:
MODEL_NAME = './data/' +  '_WindowSize'+str(args.window_size) + '_Epochs' + str(args.epochs) +'_' + str(args.input_size) + 'Sensors'
print(MODEL_NAME)

**Feature noise 제거**

feature scaling이 완료된 데이터를 EWM을 적용해 Noise를 제거해줍니다.

In [ ]:
TRAIN_DF = normalize(TRAIN_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET]).ewm(alpha=0.9).mean()

경로에 훈련된 model의 유무를 확인하고, 모델이 존재하면 훈련과정을 생략합니다.

In [ ]:
filename = MODEL_NAME + '.model'
if os.path.isfile(filename) is False:
    HS_DATASET_TRAIN = HS_DATASET(TRAIN_DF_RAW[TIMESTAMP_FIELD], TRAIN_DF,  window_size=args.window_size, stride=1)
    MODEL = model(args.input_size, args.window_size, )
    MODEL.to(args.device)
    MODEL.train()
    BEST_MODEL, LOSS_HISTORY = train(HS_DATASET_TRAIN, MODEL, args.batch_size, args.epochs)
    with open(filename, "wb") as f:
        torch.save(MODEL, f)
with open(filename, "rb") as f:
    SAVED_MODEL = torch.load(f)
    SAVED_MODEL.eval()

In [ ]:
VALIDATION_DF = normalize(VALIDATION_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET]).ewm(alpha=0.9).mean()   
HAI_DATASET_VALIDATION = HS_DATASET(VALIDATION_DF_RAW[TIMESTAMP_FIELD], VALIDATION_DF, window_size=args.window_size, attacks=VALIDATION_DF_RAW[ATTACK_FIELD])

In [ ]:
CHECK_TS, CHECK_DIST, CHECK_ATT = inference(HAI_DATASET_VALIDATION, SAVED_MODEL, args.batch_size)
ATTACK_LABELS = put_labels(np.array(VALIDATION_DF_RAW[ATTACK_FIELD]), threshold=0.5)
ANOMALY_SCORE = np.mean(CHECK_DIST, axis=1)
check_graph(ANOMALY_SCORE, CHECK_ATT, ATTACK_LABELS, piece=2, THRESHOLD=0.025, FILENAME=MODEL_NAME+'.png')

THRESHOLD에 대한 f1 점수를 텍스트화해서 저장합니다.

In [ ]:
filename = MODEL_NAME + '.txt'
fp = open(filename,'w')
# for문을 통해서 THRESHOLD 값들을 바꿔가며 가장 높게 TaR score가 나온 THRESHOLD을 선정했습니다.
# THRESHOLD가 0.01이 TaR score가 가장 높게 나왔기 때문에 최종제출에는 THRESHOLD을 0.01로 설정했습니다.
# 10초 중에 반 이상이 1일 경우 10초 모두를 1로 바꾸도록 만들었습니다.
# 공격은 최소 60초 이상 지속된다고 가정을 하고, 구간의 시작이 0이나 1일때, 같은 값이 60초보다 적으면 공격이 아니라고(0이라고) 간주했습니다.

for THRESHOLD in range(0.005,0.02,0.001):
    LABELS = put_labels(ANOMALY_SCORE, THRESHOLD)
    ATTACK_LABELS = put_labels(np.array(VALIDATION_DF_RAW[ATTACK_FIELD]), threshold=0.5)
    FINAL_LABELS = fill_blank(CHECK_TS, LABELS, np.array(VALIDATION_DF_RAW[TIMESTAMP_FIELD]))
    sec = 10
    for i in range(sec,len(FINAL_LABELS), 5):
        start = i-(sec)
        end = i
        if list(FINAL_LABELS[start:end]).count(1) > (end-start)/2:
            FINAL_LABELS[start:end] = np.array([1]*(end-start))
    sec = 60
    flag = False
    start = 0
    end = 0
    for i in range(len(FINAL_LABELS)):
        if flag == False and FINAL_LABELS[i] == 1:
            flag = True
            start = i
        if flag == True and FINAL_LABELS[i] == 0:
            end = i
            if end-start < sec:
                FINAL_LABELS[start:end] = np.array([0]*(end-start))
            flag = False

    TaPR = etapr.evaluate_haicon(anomalies=ATTACK_LABELS, predictions=FINAL_LABELS)
    print(f"THRESHOLD: {THRESHOLD:.4f} F1: {TaPR['f1']:.4f} (TaP: {TaPR['TaP']:.4f}, TaR: {TaPR['TaR']:.4f})")
    print(f"# of detected anomalies: {len(TaPR['Detected_Anomalies'])}")
    fp.write(f"THRESHOLD: {THRESHOLD:.4f} F1: {TaPR['f1']:.4f} (TaP: {TaPR['TaP']:.4f}, TaR: {TaPR['TaR']:.4f}) \n")
    fp.write(f"# of detected anomalies: {len(TaPR['Detected_Anomalies'])} \n")
fp.close()

#테스트 데이터셋 예측

앞서 진행한 데이터 전처리를 동일하게 test data에도 적용합니다. 

또한, 학습 데이터셋과 검증 데이터셋을 이용해 만든 모델로 테스트 데이터셋 결과를 예측합니다.

In [ ]:
TEST_DF = normalize(TEST_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET]).ewm(alpha=0.9).mean()
HAI_DATASET_TEST = HS_DATASET(TEST_DF_RAW[TIMESTAMP_FIELD], TEST_DF,window_size=args.window_size, attacks=None)

In [ ]:
CHECK_TS, CHECK_DIST, CHECK_ATT = inference(HAI_DATASET_TEST, SAVED_MODEL, args.batch_size)
ANOMALY_SCORE = np.mean(CHECK_DIST, axis=1)

**최종 Threshold 선정**

Threshold : 0.01로 설정하여 submission.csv파일 생성

In [ ]:
THRESHOLD = 0.01
LABELS = put_labels(ANOMALY_SCORE, THRESHOLD)
FINAL_LABELS = fill_blank(CHECK_TS, LABELS, np.array(TEST_DF_RAW[TIMESTAMP_FIELD]))

# 데이터 후처리

중간중간 noise와 같이 값이 튀는 것들이 있어, 이것들을 smoothing해주어 더 좋은 점수를 얻을 수 있었습니다.


In [ ]:
sec = 5
for i in range(sec,len(FINAL_LABELS), 3):
    start = i-(sec)
    end = i
    if list(FINAL_LABELS[start:end]).count(1) > (end-start)/2:
        FINAL_LABELS[start:end] = np.array([1]*(end-start))

공격은 최소 60초 이상 지속된다고 가정을 하고, 구간의 시작이 0이나 1일때, 같은 값이 60초보다 적으면 공격이 아니라고(0이라고) 간주했습니다.

In [ ]:
sec = 60
flag = False
start = 0
end = 0
for i in range(len(FINAL_LABELS)):
    if flag == False and FINAL_LABELS[i] == 1:
        flag = True
        start = i
    
    if flag == True and FINAL_LABELS[i] == 0:
        end = i
        if end-start < sec:
            FINAL_LABELS[start:end] = np.array([0]*(end-start))
        flag = False

예측한 결과를 제출양식에 맞춰 저장합니다.

In [ ]:
submission = pd.read_csv('./data/sample_submission.csv')
submission.index = submission['timestamp']
submission.loc[TEST_DF_RAW[TIMESTAMP_FIELD],'attack'] = FINAL_LABELS

예측한 결과를 저장하여 제출합니다.

In [ ]:
filename = MODEL_NAME + '_threshold_' + str(THRESHOLD) + '_submissionVer4.csv'
submission.to_csv(filename, index=False)